## Importing the Libraries

In [24]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Model
from keras.layers import Convolution2D,MaxPool2D,Flatten,Dense,GlobalAveragePooling2D,Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import h5py
from keras.models import model_from_json
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


## Loading images from the dataset

In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'train',
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        'valid',
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical')

Found 2682 images belonging to 7 classes.
Found 676 images belonging to 7 classes.


## Building the CNN

In [3]:
classifier=Sequential()
classifier.add(Convolution2D(filters = 64,kernel_size = (3,3),input_shape=(128, 128, 3),
                             activation='relu', data_format="channels_last"))

classifier.add(Convolution2D(filters = 128,kernel_size = (3,3), activation='relu', data_format="channels_last"))
classifier.add(MaxPool2D(pool_size=(2,2)))


classifier.add(Convolution2D(filters = 256,kernel_size = (3,3), activation='relu', data_format="channels_last"))
classifier.add(MaxPool2D(pool_size=(2,2)))


classifier.add(Flatten('channels_last'))


classifier.add(Dense(output_dim = 500, activation='relu'))
classifier.add(Dropout(0.5))

classifier.add(Dense(output_dim = 400, activation='relu'))
classifier.add(Dropout(0.5))

classifier.add(Dense(output_dim = 300, activation='relu'))
classifier.add(Dropout(0.5))

classifier.add(Dense(output_dim = train_generator.num_classes, activation='softmax'))


classifier.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['categorical_accuracy'])


## Training the Model

In [4]:
history = classifier.fit_generator(train_generator,epochs=3,
                         steps_per_epoch = 1000,
                         validation_data=validation_generator,
                         validation_steps=300,
                         use_multiprocessing=True,workers=-1)

Epoch 1/3
1000/1000 [==============================] - 6171s 6s/step - loss: 1.7208 - categorical_accuracy: 0.2829 - val_loss: 1.5322 - val_categorical_accuracy: 0.3891
Epoch 2/3
1000/1000 [==============================] - 6117s 6s/step - loss: 1.3694 - categorical_accuracy: 0.4516 - val_loss: 1.2699 - val_categorical_accuracy: 0.4789
Epoch 3/3
1000/1000 [==============================] - 7591s 8s/step - loss: 1.0120 - categorical_accuracy: 0.6246 - val_loss: 1.5637 - val_categorical_accuracy: 0.4869


## Saving the Model

In [6]:
model_json = classifier.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
classifier.save_weights("model.h5")

## Evaluating Model

In [8]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
loaded_model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['categorical_accuracy'])
score = loaded_model.evaluate(validation_generator)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
22/22 [==============================] - 30s 1s/step
categorical_accuracy: 48.67%


In [14]:
predict = loaded_model.predict(validation_generator)

In [22]:
pred_list = []
for pred in predict:
    pred_list.append(np.argmax(pred))

In [28]:
print(classification_report(validation_generator.labels,pred_list))

              precision    recall  f1-score   support

           0       0.22      0.21      0.22       113
           1       0.04      0.02      0.03        53
           2       0.16      0.21      0.18        87
           3       0.27      0.18      0.22       125
           4       0.14      0.17      0.15        89
           5       0.18      0.25      0.21       113
           6       0.16      0.12      0.14        96

    accuracy                           0.18       676
   macro avg       0.17      0.17      0.16       676
weighted avg       0.18      0.18      0.18       676

